In [1]:
import os
import copy
import joblib
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

# XGBoost (optional)
try:
    from xgboost import XGBRegressor
    HAS_XGB = True
except Exception:
    HAS_XGB = False

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader


# ---------------------------
# Helpers: metrics & splits
# ---------------------------
def rmse(y_true, y_pred):
    return float(np.sqrt(mean_squared_error(y_true, y_pred)))

def mae(y_true, y_pred):
    return float(mean_absolute_error(y_true, y_pred))

def make_splits(df):
    """Time splits."""
    train_mask = df.index < "2015-01-01"
    val_mask   = (df.index >= "2015-01-01") & (df.index < "2020-01-01")
    test_mask  = df.index >= "2020-01-01"
    return train_mask, val_mask, test_mask


# ---------------------------
# Deep learning models
# ---------------------------
class TimeSeriesDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32).view(-1, 1)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


class MLP(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
        )

    def forward(self, x):
        return self.layers(x)


class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim=64, num_layers=2):
        super().__init__()
        self.lstm = nn.LSTM(
            input_dim, hidden_dim, num_layers,
            batch_first=True, dropout=0.2
        )
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        x = x.unsqueeze(1)
        out, _ = self.lstm(x)
        out = out[:, -1, :]
        return self.fc(out)


def train_torch_model(model, train_loader, val_loader, epochs=100, lr=1e-3, patience=15):
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    best_loss = float("inf")
    best_state = None
    patience_ctr = 0

    for epoch in range(epochs):
        model.train()
        train_loss_sum = 0.0
        for Xb, yb in train_loader:
            optimizer.zero_grad()
            pred = model(Xb)
            loss = criterion(pred, yb)
            loss.backward()
            optimizer.step()
            train_loss_sum += loss.item() * Xb.size(0)

        model.eval()
        val_loss_sum = 0.0
        with torch.no_grad():
            for Xb, yb in val_loader:
                pred = model(Xb)
                loss = criterion(pred, yb)
                val_loss_sum += loss.item() * Xb.size(0)

        val_loss = val_loss_sum / len(val_loader.dataset)

        if val_loss < best_loss:
            best_loss = val_loss
            best_state = copy.deepcopy(model.state_dict())
            patience_ctr = 0
        else:
            patience_ctr += 1
            if patience_ctr >= patience:
                break

    if best_state is not None:
        model.load_state_dict(best_state)
    return model


def torch_predict_unscaled(model, X_scaled, y_scaler):
    model.eval()
    with torch.no_grad():
        preds_scaled = model(torch.tensor(X_scaled, dtype=torch.float32)).cpu().numpy()
    preds_real = y_scaler.inverse_transform(preds_scaled).reshape(-1)
    return preds_real


# ---------------------------
# Main comparison
# ---------------------------
def run_comparison():
    # Input path
    data_path = os.path.join("..", "data", "processed", "sp500_ml_ready.csv")
    
    if not os.path.exists(data_path):
        print(f"Missing: {data_path}")
        return

    df = pd.read_csv(data_path, index_col=0, parse_dates=True)

    drop_cols = ["Target_Vol", "Log_Return"]
    feature_cols = [c for c in df.columns if c not in drop_cols and ("Lag" in c or "Roll" in c)]

    X = df[feature_cols].copy()
    y = df["Target_Vol"].copy()

    train_mask, val_mask, test_mask = make_splits(df)

    X_train, y_train = X[train_mask], y[train_mask]
    X_val,   y_val   = X[val_mask],   y[val_mask]
    X_test,  y_test  = X[test_mask],  y[test_mask]

    print("Samples:", {"train": len(X_train), "val": len(X_val), "test": len(X_test)})
    print("Num features:", X.shape[1])

    # Output paths
    model_dir = os.path.join("..", "models")
    output_dir = os.path.join("..", "outputs")
    os.makedirs(model_dir, exist_ok=True)
    os.makedirs(output_dir, exist_ok=True)

    # Store results separately for RMSE and MAE
    rmse_rows = []
    mae_rows = []

    # ---------------------------
    # Classical ML
    # ---------------------------
    x_scaler = StandardScaler()
    X_train_scaled = x_scaler.fit_transform(X_train)
    X_val_scaled   = x_scaler.transform(X_val)
    X_test_scaled  = x_scaler.transform(X_test)

    joblib.dump(x_scaler, os.path.join(model_dir, "x_scaler.pkl"))

    classical_models = []
    classical_models.append(("Linear_Regression", LinearRegression(), False))
    classical_models.append(("Ridge_Regression", Ridge(alpha=1.0), False))
    classical_models.append(("Random_Forest", RandomForestRegressor(
        n_estimators=200, max_depth=10, n_jobs=-1, random_state=42
    ), False))
    classical_models.append(("KNN", KNeighborsRegressor(n_neighbors=10, weights="distance"), True))

    if HAS_XGB:
        classical_models.append(("XGBoost", XGBRegressor(
            n_estimators=300, max_depth=5, learning_rate=0.05,
            subsample=0.8, colsample_bytree=0.8, random_state=42
        ), False))
    else:
        print("XGBoost not installed; skipping XGBRegressor.")

    for name, model, use_scaled in classical_models:
        if use_scaled:
            Xtr, Xva, Xte = X_train_scaled, X_val_scaled, X_test_scaled
        else:
            Xtr, Xva, Xte = X_train, X_val, X_test

        print(f"\n--- Training {name} ---")
        model.fit(Xtr, y_train)

        pred_train = model.predict(Xtr)
        pred_test  = model.predict(Xte)

        # RMSE table
        rmse_rows.append({
            "Model": name,
            "Train": rmse(y_train, pred_train),
            "Test":  rmse(y_test, pred_test),
        })

        # MAE table
        mae_rows.append({
            "Model": name,
            "Train": mae(y_train, pred_train),
            "Test":  mae(y_test, pred_test),
        })

        joblib.dump(model, os.path.join(model_dir, f"{name}.pkl"))

    # ---------------------------
    # Deep Learning (MLP, LSTM)
    # ---------------------------
    dl_x_scaler = StandardScaler()
    X_train_dl = dl_x_scaler.fit_transform(X_train.values)
    X_val_dl   = dl_x_scaler.transform(X_val.values)
    X_test_dl  = dl_x_scaler.transform(X_test.values)

    dl_y_scaler = StandardScaler()
    y_train_dl = dl_y_scaler.fit_transform(y_train.values.reshape(-1, 1))
    y_val_dl   = dl_y_scaler.transform(y_val.values.reshape(-1, 1))

    joblib.dump(dl_x_scaler, os.path.join(model_dir, "dl_x_scaler.pkl"))
    joblib.dump(dl_y_scaler, os.path.join(model_dir, "dl_y_scaler.pkl"))

    batch_size = 64
    train_loader = DataLoader(TimeSeriesDataset(X_train_dl, y_train_dl), batch_size=batch_size, shuffle=False)
    val_loader   = DataLoader(TimeSeriesDataset(X_val_dl,   y_val_dl),   batch_size=batch_size, shuffle=False)

    input_dim = X_train.shape[1]

    dl_specs = [
        ("MLP",  MLP(input_dim)),
        ("LSTM", LSTMModel(input_dim)),
    ]

    for name, model in dl_specs:
        print(f"\n--- Training {name} ---")
        model = train_torch_model(model, train_loader, val_loader, epochs=100, lr=1e-3, patience=15)

        pred_train = torch_predict_unscaled(model, X_train_dl, dl_y_scaler)
        pred_test  = torch_predict_unscaled(model, X_test_dl,  dl_y_scaler)

        # RMSE table
        rmse_rows.append({
            "Model": name,
            "Train": rmse(y_train.values, pred_train),
            "Test":  rmse(y_test.values, pred_test),
        })

        # MAE table
        mae_rows.append({
            "Model": name,
            "Train": mae(y_train.values, pred_train),
            "Test":  mae(y_test.values, pred_test),
        })

        torch.save(model.state_dict(), os.path.join(model_dir, f"{name}.pt"))

    # ---------------------------
    # Create DataFrames
    # ---------------------------
    rmse_df = pd.DataFrame(rmse_rows).sort_values("Test").reset_index(drop=True)
    mae_df = pd.DataFrame(mae_rows).sort_values("Test").reset_index(drop=True)

    # Save to CSV
    rmse_df.to_csv(os.path.join(output_dir, "model_comparison_RMSE.csv"), index=False)
    mae_df.to_csv(os.path.join(output_dir, "model_comparison_MAE.csv"), index=False)

    # Print tables
    print("\n" + "="*50)
    print("TABLE 1: RMSE (sorted by Test)")
    print("="*50)
    print(rmse_df.to_string(index=False))

    print("\n" + "="*50)
    print("TABLE 2: MAE (sorted by Test)")
    print("="*50)
    print(mae_df.to_string(index=False))

    print(f"\nSaved: {output_dir}/model_comparison_RMSE.csv")
    print(f"Saved: {output_dir}/model_comparison_MAE.csv")

    return rmse_df, mae_df


if __name__ == "__main__":
    rmse_table, mae_table = run_comparison()


Samples: {'train': 6279, 'val': 1258, 'test': 1507}
Num features: 15

--- Training Linear_Regression ---

--- Training Ridge_Regression ---

--- Training Random_Forest ---

--- Training KNN ---

--- Training XGBoost ---

--- Training MLP ---

--- Training LSTM ---

TABLE 1: RMSE (sorted by Test)
            Model    Train     Test
              MLP 0.000361 0.000602
          XGBoost 0.000212 0.000604
    Random_Forest 0.000197 0.000607
             LSTM 0.000351 0.000618
Linear_Regression 0.000360 0.000621
              KNN 0.000000 0.000624
 Ridge_Regression 0.000380 0.000638

TABLE 2: MAE (sorted by Test)
            Model    Train     Test
          XGBoost 0.000109 0.000166
              KNN 0.000000 0.000168
             LSTM 0.000128 0.000168
    Random_Forest 0.000099 0.000175
              MLP 0.000141 0.000178
Linear_Regression 0.000142 0.000185
 Ridge_Regression 0.000157 0.000186

Saved: ../outputs/model_comparison_RMSE.csv
Saved: ../outputs/model_comparison_MAE.csv
